# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse       Shipments  \
0  Oct 26 2022 10:15AM  249406        10      0086118368   
1  Oct 26 2022  9:43AM  249402        19  ACG-2102491672   
2  Oct 26 2022  9:43AM  249402        19  ACG-2102491673   
3  Oct 26 2022  9:43AM  249402        19  ACG-2102491674   
4  Oct 26 2022  9:43AM  249402        19  ACG-2102491682   
5  Oct 26 2022  9:39AM  249400        10      0086114969   
6  Oct 26 2022  9:39AM  249400        10      0086114971   
7  Oct 26 2022  9:39AM  249400        10      0086114972   
8  Oct 26 2022  9:39AM  249400        10      0086114974   
9  Oct 26 2022  9:39AM  249400        10      0086115026   

                Customer ShipmentStatus  
0      ISDIN Corporation       Released  
1  ACG North America LLC      Executing  
2  ACG North America LLC      Executing  
3  ACG North America LLC      Executing  
4  ACG North America LLC      Executing  
5      ISDIN Corporation       Released  
6      ISDIN Corporation       Released  
7      ISDIN Corporation       Released  
8      ISDIN Corporation       Released  
9      ISDIN Corporation       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
13  249397      Executing          1
14  249398       Released         17
15  249400       Released         24
16  249402      Executing          4
17  249406       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Executing  Released
id                                 
249397                1.0       NaN
249398                NaN      17.0
249400                NaN      24.0
249402                4.0       NaN
249406                NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Executing  Released
id                                 
249302                0.0       7.0
249344                0.0       1.0
249354                0.0       1.0
249372                0.0       1.0
249382                0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Executing  Released
id                                 
249302                  0         7
249344                  0         1
249354                  0         1
249372                  0         1
249382                  0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Executing  Released
0               249302          0         7
1               249344          0         1
2               249354          0         1
3               249372          0         1
4               249382          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id Executing Released
0               249302                  7
1               249344                  1
2               249354                  1
3               249372                  1
4               249382                  1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                     Customer
0   Oct 26 2022 10:15AM  249406        10            ISDIN Corporation
1   Oct 26 2022  9:43AM  249402        19        ACG North America LLC
5   Oct 26 2022  9:39AM  249400        10            ISDIN Corporation
29  Oct 26 2022  9:37AM  249398        12                    Hush Hush
46  Oct 26 2022  9:36AM  249396        10            ISDIN Corporation
67  Oct 26 2022  9:34AM  249397        20  Adare Pharmaceuticals, Inc.
68  Oct 26 2022  9:33AM  249395        10                      Bio-PRF
76  Oct 25 2022  4:12PM  249390        21            NBTY Global, Inc.
77  Oct 25 2022  4:08PM  249389        21            NBTY Global, Inc.
78  Oct 25 2022  4:02PM  249388        21            NBTY Global, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                     Customer  \
0  Oct 26 2022 10:15AM  249406        10            ISDIN Corporation   
1  Oct 26 2022  9:43AM  249402        19        ACG North America LLC   
2  Oct 26 2022  9:39AM  249400        10            ISDIN Corporation   
3  Oct 26 2022  9:37AM  249398        12                    Hush Hush   
4  Oct 26 2022  9:36AM  249396        10            ISDIN Corporation   
5  Oct 26 2022  9:34AM  249397        20  Adare Pharmaceuticals, Inc.   
6  Oct 26 2022  9:33AM  249395        10                      Bio-PRF   
7  Oct 25 2022  4:12PM  249390        21            NBTY Global, Inc.   
8  Oct 25 2022  4:08PM  249389        21            NBTY Global, Inc.   
9  Oct 25 2022  4:02PM  249388        21            NBTY Global, Inc.   

  Executing Released  
0                  1  
1         4           
2                 24  
3                 17  
4         3       18  
5         1           
6         1        7  
7                  1  
8                  1  
9                  1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                     Customer  \
0  Oct 26 2022 10:15AM  249406        10            ISDIN Corporation   
1  Oct 26 2022  9:43AM  249402        19        ACG North America LLC   
2  Oct 26 2022  9:39AM  249400        10            ISDIN Corporation   
3  Oct 26 2022  9:37AM  249398        12                    Hush Hush   
4  Oct 26 2022  9:36AM  249396        10            ISDIN Corporation   
5  Oct 26 2022  9:34AM  249397        20  Adare Pharmaceuticals, Inc.   
6  Oct 26 2022  9:33AM  249395        10                      Bio-PRF   
7  Oct 25 2022  4:12PM  249390        21            NBTY Global, Inc.   
8  Oct 25 2022  4:08PM  249389        21            NBTY Global, Inc.   
9  Oct 25 2022  4:02PM  249388        21            NBTY Global, Inc.   

  Released Executing  
0        1            
1                  4  
2       24            
3       17            
4       18         3  
5                  1  
6        7         1  
7        1            
8        1            
9        1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse               Customer Released  \
0  Oct 26 2022 10:15AM  249406        10      ISDIN Corporation        1   
1  Oct 26 2022  9:43AM  249402        19  ACG North America LLC            
2  Oct 26 2022  9:39AM  249400        10      ISDIN Corporation       24   
3  Oct 26 2022  9:37AM  249398        12              Hush Hush       17   
4  Oct 26 2022  9:36AM  249396        10      ISDIN Corporation       18   

  Executing  
0            
1         4  
2            
3            
4         3

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


Date      id Warehouse               Customer  Released  \
0  Oct 26 2022 10:15AM  249406        10      ISDIN Corporation       1.0   
1  Oct 26 2022  9:43AM  249402        19  ACG North America LLC       NaN   
2  Oct 26 2022  9:39AM  249400        10      ISDIN Corporation      24.0   
3  Oct 26 2022  9:37AM  249398        12              Hush Hush      17.0   
4  Oct 26 2022  9:36AM  249396        10      ISDIN Corporation      18.0   

   Executing  
0        NaN  
1        4.0  
2        NaN  
3        NaN  
4        3.0

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Oct 26 2022 10:15AM  249406        10      ISDIN Corporation       1.0   
1  Oct 26 2022  9:43AM  249402        19  ACG North America LLC       0.0   
2  Oct 26 2022  9:39AM  249400        10      ISDIN Corporation      24.0   
3  Oct 26 2022  9:37AM  249398        12              Hush Hush      17.0   
4  Oct 26 2022  9:36AM  249396        10      ISDIN Corporation      18.0   

   Executing  
0        0.0  
1        4.0  
2        0.0  
3        0.0  
4        3.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10         1496295      52.0        4.0
12          249398      17.0        0.0
15          249302       7.0        0.0
19          748156       2.0        4.0
20          249397       0.0        1.0
21          997553       4.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10  1496295      52.0        4.0
1        12   249398      17.0        0.0
2        15   249302       7.0        0.0
3        19   748156       2.0        4.0
4        20   249397       0.0        1.0
5        21   997553       4.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10      52.0        4.0
1        12      17.0        0.0
2        15       7.0        0.0
3        19       2.0        4.0
4        20       0.0        1.0
5        21       4.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   52.0
1        12  Released   17.0
2        15  Released    7.0
3        19  Released    2.0
4        20  Released    0.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    12   15   19   20   21
Status                                   
Executing   4.0   0.0  0.0  4.0  1.0  0.0
Released   52.0  17.0  7.0  2.0  0.0  4.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    12   15   19   20   21
0          Executing   4.0   0.0  0.0  4.0  1.0  0.0
1           Released  52.0  17.0  7.0  2.0  0.0  4.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    12   15   19   20   21
0  Executing   4.0   0.0  0.0  4.0  1.0  0.0
1   Released  52.0  17.0  7.0  2.0  0.0  4.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()